In [1]:
import datetime
import math
import time
import itertools
from contextlib import closing
from collections import OrderedDict
import json
import numpy as np
import pandas as pd
import MySQLdb
import pymssql
from sqlalchemy import create_engine
import dfutils

In [2]:
#DB_NAME = 'wic'
#WIC_DB_HOST = 'wic-risk-database.cwi02trt7ww1.us-east-1.rds.amazonaws.com'
#DB_USER = 'aduprey'
#DB_PASSWORD = 'aduprey'
WIC_DB_HOST = 'wic-risk-database.cwi02trt7ww1.us-east-1.rds.amazonaws.com'
DB_USER = 'root'
DB_PASSWORD = 'waterislandcapital'
DB_NAME = 'wic'
engine = create_engine("mysql://" + DB_USER + ":" + DB_PASSWORD + "@" + WIC_DB_HOST + "/wic")

In [3]:
def wic_optimized_execute(query, commit=False, retrieve_column_names=False, connection_timeout=250, extra_values=None):
    with closing(MySQLdb.connect(host=WIC_DB_HOST, user=DB_USER, passwd=DB_PASSWORD, db=DB_NAME)) as wic_cnx:
        with closing(wic_cnx.cursor()) as wic_cursor:

            if extra_values is not None:
                wic_cursor.execute(query, (extra_values,))
            else:
                wic_cursor.execute(query)
            if commit:
                wic_cnx.commit()
                return
            fetched_res = wic_cursor.fetchall()  # fetch (and discard) remaining rows

            if retrieve_column_names:
                return fetched_res, [i[0] for i in wic_cursor.description]

            return fetched_res


In [4]:
def tradar_optimized_execute(query, commit=False):
    try:
        with closing(pymssql.connect(host='NYDC-WTRTRD01', user='paz', password='Welcome2',
                                     database='TradarBE')) as trdr_pnl_conn:
            with closing(trdr_pnl_conn.cursor()) as trdr_pnl_cursor:
                trdr_pnl_cursor.execute(query)
                if commit:
                    trdr_pnl_conn.commit()
                    return

                fetched_res = trdr_pnl_cursor.fetchall()  # fetch (and discard) remaining rows\
                return fetched_res
    except Exception as e:
        print e

In [5]:
def get_active_tradegroups_from_tradar():
    query = "SELECT DISTINCT " \
            "MAX(A.timeKey) as MaxDate, " \
            "F.fund AS Fund, " \
            "B.strat AS TradeGroup, " \
            "S.ticker, " \
            "ST.sectype, " \
            "ST.secTypeDesc " \
            "FROM performanceAttributionFact AS A " \
            "INNER JOIN TradeStrat AS B ON A.stratKey = B.stratId " \
            "INNER JOIN TradeFund AS F ON A.fundKey = F.fundId " \
            "INNER JOIN Sec AS S ON S.secId = A.secIdKey " \
            "INNER JOIN dbo.SecType ST on S.sectype = ST.sectype " \
            "WHERE A.timeKey = (SELECT MAX(timekey) FROM dbo.PerformanceAttributionFact WHERE timekey < (SELECT max(timekey) FROM dbo.PerformanceAttributionFact)) " \
            "AND ST.secTypeDesc NOT IN ('Exchrate') " \
            "AND F.fund IN ('ARB', 'AED', 'TACO', 'WED', 'CAM', 'LG', 'TAQ', 'LEV', 'MACO', 'MALT') " \
            "GROUP BY A.timeKey, B.strat, F.fund, S.ticker, ST.sectype, ST.secTypeDesc "
    
    cols = ['Date', 'Fund', 'TradeGroup', 'Ticker', 'SecType', 'SecTypeDesc']
    query_result = tradar_optimized_execute(query)
    df = pd.DataFrame(query_result, columns=cols)
    unique_combos_df = df.drop_duplicates(['Fund', 'TradeGroup'])
    return unique_combos_df.reset_index(drop=True)

In [6]:
a = get_active_tradegroups_from_tradar().reset_index()

In [7]:
len(a)

593

In [8]:
def get_tradegroup_funds_from_overall_pnl():
    query = "SELECT DISTINCT " \
            "MAX(`Date`) as MaxDate, " \
            "Fund, " \
            "TradeGroup " \
            "FROM wic.tradegroup_overall_pnl " \
            "WHERE `Date` = (SELECT MAX(`Date`) FROM wic.tradegroup_overall_pnl) " \
            "GROUP BY `Date`, Fund, TradeGroup "
    
    cols = ['Date', 'Fund', 'TradeGroup']
    query_results = wic_optimized_execute(query)
    df = pd.DataFrame(list(query_results), columns=cols)
    return df

In [9]:
b = get_tradegroup_funds_from_overall_pnl()

In [10]:
len(b)

593

In [11]:
a['TradeGroup'] = a['TradeGroup'].apply(lambda x: x.upper())
b['TradeGroup'] = b['TradeGroup'].apply(lambda x: x.upper())

In [14]:
pd.merge(a, b, how='right', on=['Fund', 'TradeGroup'])

,index,Date_x,Fund,TradeGroup,Ticker,SecType,SecTypeDesc,Date_y
0,0,20190515,AED,AABA R/R SHORT,AABA US,46,Equity Investment Trust,2019-05-15
1,1,20190515,CAM,AABA R/R SHORT,AABA US,46,Equity Investment Trust,2019-05-15
2,2,20190515,LG,AABA R/R SHORT,AABA US,46,Equity Investment Trust,2019-05-15
3,3,20190515,TAQ,AABA R/R SHORT,AABA US,46,Equity Investment Trust,2019-05-15
4,4,20190515,WED,AABA R/R SHORT,AABA US,46,Equity Investment Trust,2019-05-15
5,5,20190515,TAQ,ACOR R/R,ACOR 190517C00011000,62,Equity Call,2019-05-15
6,6,20190515,AED,ADSW - WM,ADSW US,50,Equity,2019-05-15
7,7,20190515,ARB,ADSW - WM,ADSW US,50,Equity,2019-05-15
8,8,20190515,CAM,ADSW - WM,ADSW US,50,Equity,2019-05-15
9,9,20190515,LEV,ADSW - WM,ADSW US,50,Equity,2019-05-15


In [105]:
def get_tradegroup_funds_from_front_page():
    query = "SELECT DISTINCT " \
            "MAX(`Date`) as MaxDate, " \
            "Fund, " \
            "TradeGroup, " \
            "Status " \
            "FROM wic.tradegroup_attribution_to_fund_nav_bps " \
            "WHERE `Date` = (SELECT MAX(`Date`) FROM wic.tradegroup_attribution_to_fund_nav_bps) " \
            "AND Status = 'CLOSED' " \
            "GROUP BY `Date`, Fund, TradeGroup "
    
    cols = ['Date', 'Fund', 'TradeGroup', 'Status']
    query_results = wic_optimized_execute(query)
    df = pd.DataFrame(list(query_results), columns=cols)
    return df

In [106]:
c = get_tradegroup_funds_from_front_page()

In [107]:
len(c)

640

In [13]:
pd.set_option('display.max_rows', 1500)
pd.set_option('display.max_columns', 50)

In [104]:
a['TradeGroup'] = a['TradeGroup'].apply(lambda x: x.upper())
b['TradeGroup'] = b['TradeGroup'].apply(lambda x: x.upper())
pd.merge(a, c, how='outer', on=['Fund', 'TradeGroup'])

,Date_x,Fund,TradeGroup,Ticker,SecType,SecTypeDesc,Date_y,Status
0,20190514.0,AED,AABA R/R SHORT,AABA US,46.0,Equity Investment Trust,2019-05-14,ACTIVE
1,20190514.0,CAM,AABA R/R SHORT,AABA US,46.0,Equity Investment Trust,2019-05-14,ACTIVE
2,20190514.0,LG,AABA R/R SHORT,AABA US,46.0,Equity Investment Trust,2019-05-14,ACTIVE
3,20190514.0,TAQ,AABA R/R SHORT,AABA US,46.0,Equity Investment Trust,2019-05-14,ACTIVE
4,20190514.0,WED,AABA R/R SHORT,AABA US,46.0,Equity Investment Trust,2019-05-14,ACTIVE
5,20190514.0,TAQ,ACOR R/R,ACOR 190517C00011000,62.0,Equity Call,2019-05-14,ACTIVE
6,20190514.0,AED,ADSW - WM,ADSW US,50.0,Equity,2019-05-14,ACTIVE
7,20190514.0,ARB,ADSW - WM,ADSW US,50.0,Equity,2019-05-14,ACTIVE
8,20190514.0,CAM,ADSW - WM,ADSW US,50.0,Equity,2019-05-14,ACTIVE
9,20190514.0,LEV,ADSW - WM,ADSW US,50.0,Equity,2019-05-14,ACTIVE
